In [57]:
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import datetime

In [58]:
pred_model = pd.read_csv("Pred_Model.csv")

In [59]:
pred_model['start_time'] = pd.to_datetime(pred_model['start_time'])
pred_model['end_time'] = pd.to_datetime(pred_model['end_time'])


In [60]:
pred_model_counts = pred_model.groupby([pd.Grouper(key='start_time', freq = 'H')]).size().to_frame('counts')

In [61]:
pred_model_counts["counts"].sum()
#pred_model_counts.head()

518963

In [62]:
# make index to column to create day of week and month based on date
pred_model_counts['index1'] = pred_model_counts.index
pred_model_counts["weekday"] = pred_model_counts["index1"].dt.dayofweek
pred_model_counts["month"] = pred_model_counts["index1"].dt.month

In [ ]:
pred_model_counts = pred_model_counts.drop('index1', axis=1)

KeyError: "['index1'] not found in axis"

In [92]:
pred_model_counts.index.names = ['daytime']
pred_model_counts.head()
pred_model_counts.tail()
#pred_model_counts.info()

,counts,weekday,month
daytime,,,
2016-12-31 19:00:00,13,5,12
2016-12-31 20:00:00,13,5,12
2016-12-31 21:00:00,21,5,12
2016-12-31 22:00:00,4,5,12
2016-12-31 23:00:00,6,5,12


## Import weather and merge with pred_model_counts

In [81]:
weather_hourly = pd.read_csv('weather_hourly.csv')
weather_hourly.head()

,Unnamed: 0,max_temp,min_temp,precip,interp_maxtemp,interp_mintemp,interp_precip,update_maxtemp,update_mintemp,update_precip
0,2015-01-01 06:00:00,-5.6,-5.6,0.0,-5.6,-5.6,0.0,-5.6,-5.6,0.0
1,2015-01-01 07:00:00,-5.6,-5.6,0.0,-5.6,-5.6,0.0,-5.6,-5.6,0.0
2,2015-01-01 08:00:00,-5.6,-5.6,0.0,-5.6,-5.6,0.0,-5.6,-5.6,0.0
3,2015-01-01 09:00:00,-4.4,-4.4,0.0,-4.4,-4.4,0.0,-4.4,-4.4,0.0
4,2015-01-01 10:00:00,-5.6,-5.6,0.0,-5.6,-5.6,0.0,-5.6,-5.6,0.0


In [82]:
# Creating Dataframe only containing values from 2016 
# weatherhour = weather_hourly.loc['2016-01-01 00:00:00' : '2016-12-31 23:00:00']
# weatherhour['temp_av'] = (weatherhour['update_maxtemp'] + weatherhour['update_mintemp']) / 2
# weatherhour = weatherhour.drop(columns = ['update_mintemp', 'update_maxtemp'])
weatherhour.head()

,update_precip,temp_av
daytime,,
2016-01-01 00:00:00,0.0,5.6
2016-01-01 01:00:00,0.0,5.0
2016-01-01 02:00:00,0.0,5.0
2016-01-01 03:00:00,0.0,4.4
2016-01-01 04:00:00,0.0,4.4


In [83]:
weatherhour = weather_hourly.drop(columns = ['max_temp','min_temp','precip','interp_maxtemp','interp_mintemp','interp_precip'])
weatherhour['temp_av'] = (weatherhour['update_maxtemp'] + weatherhour['update_mintemp']) / 2
weatherhour = weatherhour.drop(columns = ['update_mintemp', 'update_maxtemp'])
weatherhour = weatherhour.set_index('Unnamed: 0')
weatherhour.index.names = ['daytime']
weatherhour.head(5)


,update_precip,temp_av
daytime,,
2015-01-01 06:00:00,0.0,-5.6
2015-01-01 07:00:00,0.0,-5.6
2015-01-01 08:00:00,0.0,-5.6
2015-01-01 09:00:00,0.0,-4.4
2015-01-01 10:00:00,0.0,-5.6


In [91]:
weatherhour = weatherhour.loc['2016-01-01 00:00:00' : '2016-12-31 23:00:00']
weatherhour.head()
weatherhour.tail()

,update_precip,temp_av
daytime,,
2016-12-31 19:00:00,0.0,1.7
2016-12-31 20:00:00,0.0,1.7
2016-12-31 21:00:00,0.0,1.7
2016-12-31 22:00:00,0.0,2.2
2016-12-31 23:00:00,0.0,3.3


In [88]:
# pred_model_final = pd.merge(pred_model_counts, weatherhour, left_index=True, right_index=True)
# pred_model_final.head()
# weatherhour.join(pred_model_final)
pd.concat([pred_model_counts, weatherhour], axis=1)


,counts,weekday,month,update_precip,temp_av
daytime,,,,,
2016-01-01 00:00:00,7.0,4.0,1.0,NaN,NaN
2016-01-01 01:00:00,6.0,4.0,1.0,NaN,NaN
2016-01-01 02:00:00,3.0,4.0,1.0,NaN,NaN
2016-01-01 03:00:00,1.0,4.0,1.0,NaN,NaN
2016-01-01 04:00:00,0.0,4.0,1.0,NaN,NaN
...,...,...,...,...,...
2016-12-31 19:00:00,NaN,NaN,NaN,0.0,1.7
2016-12-31 20:00:00,NaN,NaN,NaN,0.0,1.7
2016-12-31 21:00:00,NaN,NaN,NaN,0.0,1.7


- Bike data indexieren
- bereinigen
- groupby.dt.hour.size()
- mergen
- Holdout/Training Split

- Time of the day in hours 
- Month
- Weekday (0-6)
- Temperature (average of min and max temp cause very similar)
- Precipitation (0,1)

In [ ]:
Polynomial (lasso/...)
Random Forest
Neural Networks